# Get CarGurus Expected Price

In [1]:
import requests
import pandas as pd
import os
import numpy as np

In [2]:
# Read in scraped car data
data_file = os.path.join(os.getcwd(),"data","all_cars.csv")  # change to updated scraped data file
cars = pd.read_csv(data_file)
pd.set_option("display.max_columns",None) 
cars

,post_date,lastpull_ts,link,price,year,make,model,body,mileage,title_type,city,state,seller,trim,ext_color,int_color,transmission,liters,cylinders,fuel_type,n_doors,ext_condition,int_condition,drive_type,VIN,n_pics
0,2020-03-27,1585374184,https://cars.ksl.com/listing/6204647,8995,2014.0,Ford,Focus,Hatchback,111050.0,Clean Title,St. George,UT,Dealer,Titanium,black,black,Automatic,NaN,4.0,Flex Fuel,NaN,NaN,NaN,FWD,1FADP3N24EL463439,12
1,2020-03-27,1585374185,https://cars.ksl.com/listing/6253659,10995,2015.0,Toyota,Corolla,Sedan,55044.0,Clean Title,Lindon,UT,Dealer,S Plus,white,black,Automatic,NaN,4.0,Gasoline,NaN,NaN,NaN,FWD,5YFBURHE2FP238514,48
2,2020-03-27,1585374186,https://cars.ksl.com/listing/6320317,34869,2019.0,Jeep,Grand Cherokee,Sport Utility,22192.0,NaN,Boise,ID,Dealer,Limited,diamond black crystal pearlcoat,black,Automatic,NaN,6.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1C4RJFBG8KC812665,27
3,2020-03-27,1585374187,https://cars.ksl.com/listing/6203357,38495,2017.0,Chevrolet,Silverado 1500,Truck,21747.0,NaN,Twin Falls,ID,Dealer,LTZ,gray,jet black,Automatic,NaN,8.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,3GCUKSEC2HG500663,58
4,2020-03-27,1585374188,https://cars.ksl.com/listing/6225558,6199,2007.0,Mitsubishi,Raider,Truck,119643.0,Clean Title,St. George,UT,Dealer,LS,electric blue,NaN,Manual,NaN,6.0,Gasoline,NaN,NaN,NaN,RWD,1Z7HC22K87S213334,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34451,2020-03-20,1584833057,https://cars.ksl.com/listing/6269151,20148,2019.0,Ford,Fusion Hybrid,Sedan,20303.0,NaN,Price,UT,Dealer,SE,silver,NaN,NaN,NaN,4.0,Hybrid,NaN,NaN,NaN,FWD,3FA6P0LU0KR206951,0
34452,2020-03-20,1584833058,https://cars.ksl.com/listing/5968149,15995,2009.0,Jeep,Grand Cherokee,Sport Utility,122000.0,NaN,NaN,NaN,Dealer,SRT-8,red,charcoal gray,Automatic,NaN,8.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1J8HR78W89C505718,19
34453,2020-03-20,1584833058,https://cars.ksl.com/listing/6287241,52900,2020.0,Acura,MDX,Sport Utility,2080.0,NaN,Salt Lake City,UT,Dealer,with Advance/Entertainment Package,white,black,Automatic,NaN,6.0,Gasoline,NaN,NaN,NaN,AWD,5J8YD4H99LL011505,50
34454,2020-03-20,1584833059,https://cars.ksl.com/listing/6269149,22325,2019.0,Hyundai,Santa Fe,Sport Utility,17195.0,NaN,Price,UT,Dealer,SE,quartz white,espresso/gray,Automatic,NaN,4.0,Gasoline,NaN,NaN,NaN,AWD,5NMS2CAD0KH106182,21


# Create Zip Codes

In [3]:
# Create dictionary of zip codes for all towns to look up expected price
# Website to look up most zipcodes
# http://localistica.com/usa/ut/salt%20lake%20city/zipcodes/all-zipcodes/

import bs4
def get_most_populated_zip_code(city):
    city, state = city.split(",")[0], city.split(",")[1].strip().lower()
    r = requests.get(f'http://localistica.com/usa/{state}/{city.replace(" ", "%20")}/zipcodes/most-populated-zipcodes/')
    bsoup = bs4.BeautifulSoup(r.text)
    try:
        return bsoup.find(id="dgZipCodes").find_all("tr")[1].td.a.text
    except:
        return None
    

# get unique cities in dataframe
all_cities = [car for car in cars.city.unique() if type(car) == str]
# keyList = [x + ", " + cars[cars.city == x].iloc[0]['state'] for x in all_cities]
# look up zipcode
zip_codes = {key: get_most_populated_zip_code(key + ", " + cars[cars.city == key].iloc[0]['state']) for key in all_cities}
# hard code the ones it missed
zip_codes.update({'West Valley City': 84118, 'St. George': 84770, 'Taylorsville': 84123, 'Gunnison': 84622,
                 'Riverdale': 84405, 'Murray': 84107, 'Clinton': 84015, 'Perry': 84302})
# check for missing zip codes
print(len([k for k,v in zip_codes.items() if v == None]))

96


In [4]:
zip_codes

{'St. George': 84770,
 'Lindon': '84042',
 'Boise': '83709',
 'Twin Falls': '83301',
 'Ogden': '84404',
 'Layton': '84041',
 'Draper': '84020',
 'Murray': 84107,
 'Orem': '84057',
 'Salt Lake City': '84120',
 'Provo': '84604',
 'Farmington': '84025',
 'Centerville': '84014',
 'Midvale': '84047',
 'Sandy': '84092',
 'Morgan': '84050',
 'Millcreek': None,
 'Rexburg': '83440',
 'Springville': '84663',
 'West Valley City': 84118,
 'American Fork': '84003',
 'Elko': '89801',
 'Delta': '84624',
 'Bountiful': '84010',
 'Spanish Fork': '84660',
 'Riverton': '84065',
 'Pleasant Grove': '84062',
 'Brigham City': '84302',
 'Kaysville': '84037',
 'Woods Cross': '84087',
 'South Jordan': '84095',
 'Lehi': '84043',
 'Nephi': '84648',
 'Magna': '84044',
 'Malad City': '83252',
 'Pocatello': '83201',
 'Manti': '84642',
 'Roosevelt': '84066',
 'Highland': None,
 'Burley': '83318',
 'Lewiston': '84320',
 'North Salt Lake': '84054',
 'Logan': '84321',
 'Idaho Falls': '83401',
 'Herriman': '84096',
 'Blac

# CarGurus Pricing
Terms of Use: https://www.cargurus.com/Cars/TermsOfUse.html

Using car make, model, mileage, year, and zip code of listing to get the privatelisting value from CarGurus to compare listed prices to an expected price

In [153]:
# Need to edit, just seeing if it works

# list of all cars in CarGurus database
all_cars = requests.get("https://www.cargurus.com/Cars/getCarPickerReferenceDataAJAX.action?showInactive=false&useInventoryService=false&quotableCarsOnly=false&localCountryCarsOnly=true&outputFormat=REACT").json()

# gets CarGuru make and model id to find price for individual make and model
def get_cargurus_maker_and_model_ids(all_cars, car_make, car_model):
    try:
        all_models = [x for x in all_cars.get('allMakerModels').get('makers') if x.get('name') == car_make][0]
        return (all_models.get('id'), [x for x in all_models.get('models') if x.get('name') == car_model][0].get('id'))
    except (IndexError, AttributeError):
        return (None, None)
    
# gets the entity id which includes the make, model, and year of car
def get_entity_id(maker_id, model_id, car_year):
    try:
        all_entities = requests.get(f"https://www.cargurus.com/Cars/getSelectedMakerModelCarsAJAX.action?showInactive=false&useInventoryService=false&quotableCarsOnly=false&localCountryCarsOnly=true&outputFormat=REACT&maker={maker_id}").json()
        model_entity_ids = [car for car in all_entities.get('models') if car.get('id') == model_id][0]
        return [ids for ids in model_entity_ids.get('cars') if ids.get('year') == car_year][0].get('id')
    except (IndexError, AttributeError):
        return None
    
# gets the estimated listing price of the car based on entity id and the mileage
def get_price(car_make, car_model, car_year, car_mileage, car_zip_code, all_cars):
    maker_id, model_id = get_cargurus_maker_and_model_ids(all_cars, car_make, car_model)
    if not model_id:
        return None
    
    entity_id = get_entity_id(maker_id, model_id, car_year)
    
    if not entity_id:
        return None 
    # data needed to request CarGurus report
    data = {
        'carDescription.radius': 75,
        'selectedEntity': entity_id,
        'carDescription.transmission': "",
        'carDescription.mileage': car_mileage,
        'carDescription.postalCode': car_zip_code,
        'carDescription.engineId': "",
        'carDescription.vin': "",
        'carDescription.vinType': "",
        'forPrivateListing': True,
        'inventoryListingId' : ""
    }
    
    res = requests.post("https://www.cargurus.com/Cars/generateReportJsonAjax.action", data=data)
    return res.json().get("priceDetails").get("privateListingPrice") #private listing price from CarGurus report


for index, row in cars.iterrows():
    if row["city"] in zip_codes.keys():
        expected_price = get_price(row["make"], row["model"], row["year"], row["mileage"], zip_codes.get(row["city"]), all_cars)
        # change expected prices that are 0 to none
        if expected_price == None or expected_price < 1:
            expected_price = None 
        cars.loc[index, "expected_price"] = expected_price
    else:
        cars.loc[index, "expected_price"] = None
        

In [154]:
cars

,post_date,lastpull_ts,link,price,year,make,model,body,mileage,title_type,city,state,seller,trim,ext_color,int_color,transmission,liters,cylinders,fuel_type,n_doors,ext_condition,int_condition,drive_type,VIN,n_pics,expected_price
0,2020-03-21,1584801100,https://cars.ksl.com/listing/6107041,35000,2015,Tesla,Model S,Sedan,21139,NaN,Salt Lake City,UT,Dealer,kWh Battery,black,tan,Automatic,NaN,NaN,Electric,4.0,NaN,NaN,RWD,5YJSA1E14FF116177,67,42446.0
1,2020-03-21,1584801100,https://cars.ksl.com/listing/6220811,16999,2014,GMC,1500,Truck,91709,NaN,Woods Cross,UT,Dealer,SLE,summit white,cocoa/dune,Automatic,NaN,8.0,Flex Fuel,4.0,NaN,NaN,4-Wheel Drive,1GTV2UEC5EZ263713,31,NaN
2,2020-03-21,1584801101,https://cars.ksl.com/listing/6220809,26978,2016,Infiniti,Q70L,Sedan,7042,NaN,Woods Cross,UT,Dealer,NaN,majestic white,wheat,Automatic,NaN,8.0,Gasoline,4.0,NaN,NaN,AWD,JN1AY1PR5GM590046,31,NaN
3,2020-03-21,1584801102,https://cars.ksl.com/listing/6289112,9500,2015,Hyundai,Elantra GT,Compact,79199,Clean Title,Price,UT,Owner,NaN,blue,gray,Automatic,NaN,4.0,Gasoline,4.0,NaN,Very Good,FWD,KMHD35LH9FU242533,13,8243.0
4,2020-03-21,1584801103,https://cars.ksl.com/listing/5952682,34335,2018,Jeep,Wrangler Unlimited,Convertible,25,NaN,West Valley City,UT,Dealer,Sport S,gobi clearcoat,black,Automatic,NaN,6.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1C4HJWDG6JL898663,20,32140.0
5,2020-03-21,1584801106,https://cars.ksl.com/listing/5952680,34335,2018,Jeep,Wrangler Unlimited,Convertible,10,NaN,West Valley City,UT,Dealer,Sport S,black clearcoat,black,Automatic,NaN,6.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1C4HJWDG7JL856342,20,32142.0
6,2020-03-21,1584801107,https://cars.ksl.com/listing/6220805,22199,2017,Ford,Edge,Sport Utility,3693,NaN,Woods Cross,UT,Dealer,Titanium,ruby red metallic tinted clearcoat,ebony,Automatic,NaN,4.0,Gasoline,4.0,NaN,NaN,AWD,2FMPK4K98HBB73475,31,23821.0
7,2020-03-21,1584801107,https://cars.ksl.com/listing/5952679,34335,2018,Jeep,Wrangler Unlimited,Convertible,20,NaN,West Valley City,UT,Dealer,Sport S,bright white clearcoat,black,Automatic,NaN,6.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1C4HJWDG8JL856348,21,32141.0
8,2020-03-21,1584801108,https://cars.ksl.com/listing/6220804,12399,2016,Subaru,Impreza,Hatchback,35458,NaN,Woods Cross,UT,Dealer,2.0i Sport Limited,dark gray metallic,black,Automatic,NaN,4.0,Gasoline,4.0,NaN,NaN,NaN,JF1GPAY67G8229916,31,13287.0
9,2020-03-21,1584801110,https://cars.ksl.com/listing/6220802,28999,2017,Acura,MDX,Sport Utility,12101,NaN,Woods Cross,UT,Dealer,with Advance Package,crystal black pearl,ebony,Automatic,NaN,6.0,Gasoline,4.0,NaN,NaN,AWD,5FRYD4H83HB013622,31,30056.0
